# Market Basket Analysis

In [2]:
!pip install seaborn

   ---------------------------------------- 0.0/294.9 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/294.9 kB 1.3 MB/s eta 0:00:01
   ------------------- -------------------- 143.4/294.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 294.9/294.9 kB 3.0 MB/s eta 0:00:00


In [11]:
!pip install apyori 

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for apyori: filename=apyori-1.1.2-py3-none-any.whl size=5976 sha256=b9d633395fb89a3decc2a7442206c827f01c96054c78bda4bcac1cefb6317d2a
  Stored in directory: c:\users\farah\appdata\local\pip\cache\wheels\77\3d\a6\d317a6fb32be58a602b1e8c6b5d6f31f79322da554cad2a5ea
Successfully built apyori


In [4]:
#Packages
import numpy as np # linear algebra
import pandas as pd # Data pre-processing
import seaborn as sns # Required for plotting
import matplotlib.pyplot as plt # Required for plotting

## Apriori Algorithm


### Consumer

In [56]:
df_dataco = pd.read_csv('DataCoSupplyChainDataset.csv', encoding="latin1", sep=";")

In [57]:
p_df = df_dataco[df_dataco['Customer Segment'] == 'Consumer']
p_df

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Id,Customer Segment,Department Name,...,Order Item Quantity,Sales,Order Item Total,Order Region,Order State,Order Status,Product Name,Product Price,shipping date (DateOrders),Shipping Mode
0,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,20755,Consumer,Fitness,...,1,327.750000,314.640015,Southeast Asia,Java Occidental,COMPLETE,Smart watch,327.750000,02/03/2018 22:56,Standard Class
1,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,19492,Consumer,Fitness,...,1,327.750000,311.359985,South Asia,Rajastán,PENDING,Smart watch,327.750000,1/18/2018 12:27,Standard Class
2,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,19491,Consumer,Fitness,...,1,327.750000,309.720001,South Asia,Rajastán,CLOSED,Smart watch,327.750000,1/17/2018 12:06,Standard Class
5,6,4,18.580000,294.980011,Shipping canceled,0,Sporting Goods,19488,Consumer,Fitness,...,1,327.750000,294.980011,Oceania,Queensland,CANCELED,Smart watch,327.750000,1/19/2018 11:03,Standard Class
11,5,2,45.689999,268.760010,Late delivery,1,Sporting Goods,19482,Consumer,Fitness,...,1,327.750000,268.760010,Southeast Asia,Célebes Septentrional,PENDING,Smart watch,327.750000,1/18/2018 8:57,Second Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180500,3,4,138.229996,383.980011,Advance shipping,0,Fishing,773,Consumer,Fan Shop,...,1,399.980011,383.980011,Oceania,Queensland,COMPLETE,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/19/2016 15:35,Standard Class
180507,5,2,50.110001,347.980011,Late delivery,1,Fishing,8247,Consumer,Fan Shop,...,1,399.980011,347.980011,Southeast Asia,Th? Dô Hà N?i,PAYMENT_REVIEW,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/21/2016 9:16,Second Class
180511,2,2,157.429993,327.980011,Shipping on time,0,Fishing,9314,Consumer,Fan Shop,...,1,399.980011,327.980011,Eastern Asia,Sichuan,ON_HOLD,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/18/2016 6:28,Second Class
180517,3,4,186.229996,387.980011,Advance shipping,0,Fishing,2813,Consumer,Fan Shop,...,1,399.980011,387.980011,Oceania,Australia del Sur,PENDING_PAYMENT,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/18/2016 20:18,Standard Class


In [58]:
cust_level_p = p_df[["Customer Id", "Product Name"]].sort_values(by = "Customer Id", ascending = False) ## Selecting only required variables for modelling
cust_level_p['Product Name'] = cust_level_p['Product Name'].str.strip() # Removing white spaces if any
cust_level_p

,Customer Id,Product Name
0,20755,Smart watch
93905,20754,Summer dresses
160537,20753,Summer dresses
18819,20752,Summer dresses
125524,20747,Summer dresses
...,...,...
106166,2,O'Brien Men's Neoprene Life Vest
144126,2,O'Brien Men's Neoprene Life Vest
95017,2,Diamondback Women's Serene Classic Comfort Bi
94974,2,Nike Men's Dri-FIT Victory Golf Polo


In [59]:
transactions_p = [a[1]['Product Name'].tolist() for a in list(cust_level_p.groupby(['Customer Id']))] ## Combing all the items in list format for each cutomer

In [60]:
transactions_p

[["Nike Men's Free 5.0+ Running Shoe"],
 ['Pelican Sunstream 100 Kayak',
  'Pelican Sunstream 100 Kayak',
  "Under Armour Girls' Toddler Spine Surge Runni",
  'Perfect Fitness Perfect Rip Deck',
  'Field & Stream Sportsman 16 Gun Fire Safe',
  "O'Brien Men's Neoprene Life Vest",
  "O'Brien Men's Neoprene Life Vest",
  "O'Brien Men's Neoprene Life Vest",
  "Diamondback Women's Serene Classic Comfort Bi",
  "Nike Men's Dri-FIT Victory Golf Polo"],
 ["Diamondback Women's Serene Classic Comfort Bi",
  'Pelican Sunstream 100 Kayak',
  "Under Armour Kids' Mercenary Slide",
  "Nike Men's Dri-FIT Victory Golf Polo",
  'Perfect Fitness Perfect Rip Deck',
  "Diamondback Women's Serene Classic Comfort Bi",
  "Nike Men's Free 5.0+ Running Shoe",
  "Nike Men's Dri-FIT Victory Golf Polo",
  "Nike Men's Dri-FIT Victory Golf Polo",
  "O'Brien Men's Neoprene Life Vest",
  'Perfect Fitness Perfect Rip Deck',
  'Perfect Fitness Perfect Rip Deck',
  'Field & Stream Sportsman 16 Gun Fire Safe',
  'Pelican 

In [61]:
#Importing and Setting the apriori options
#1. Minimum Support
#2. Minimum Confidence
#3. Minimum Lift
#4. Lenght = the number of items

In [62]:
from apyori import apriori ## Importing apriori package
rules = apriori(transactions = transactions_p,min_support = 0.01, min_confidence = 0.5, min_lift = 1.12 , min_length = 2, max_length = 2) ## Model Creation

In [63]:
results = list(rules)

In [64]:
## Creating user-defined function for arranging the results obtained from model into readable format

def inspect(results):
    x         = [tuple(result[2][0][0])[0] for result in results]
    y         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(x, y, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['X', 'Y', 'Support', 'Confidence', 'Lift'])

In [69]:
resultsinDataFrame.nlargest(n=40, columns='Support') ## Showing best possible scenarios

,X,Y,Support,Confidence,Lift
165,Nike Men's CJ Elite 2 TD Football Cleat,Perfect Fitness Perfect Rip Deck,0.395792,0.840048,1.724436
203,Nike Men's Dri-FIT Victory Golf Polo,Perfect Fitness Perfect Rip Deck,0.387564,0.841624,1.727672
241,O'Brien Men's Neoprene Life Vest,Perfect Fitness Perfect Rip Deck,0.377560,0.848676,1.742148
159,Nike Men's CJ Elite 2 TD Football Cleat,Nike Men's Dri-FIT Victory Golf Polo,0.375877,0.797777,1.732432
163,Nike Men's CJ Elite 2 TD Football Cleat,O'Brien Men's Neoprene Life Vest,0.361103,0.766422,1.722758
87,Field & Stream Sportsman 16 Gun Fire Safe,Perfect Fitness Perfect Rip Deck,0.360449,0.844839,1.734271
201,Nike Men's Dri-FIT Victory Golf Polo,O'Brien Men's Neoprene Life Vest,0.358392,0.778274,1.749399
78,Field & Stream Sportsman 16 Gun Fire Safe,Nike Men's CJ Elite 2 TD Football Cleat,0.349977,0.820294,1.741028
81,Field & Stream Sportsman 16 Gun Fire Safe,Nike Men's Dri-FIT Victory Golf Polo,0.344086,0.806487,1.751346
264,Pelican Sunstream 100 Kayak,Perfect Fitness Perfect Rip Deck,0.342964,0.846722,1.738137


In [27]:
#result1 = resultsinDataFrame[resultsinDataFrame['X'] == 'Health&Beauty D'] #select highh price
#result1.nlargest(n=5, columns='Support')


In [18]:
results

[RelationRecord(items=frozenset({'Bag Boy Beverage Holder', "Nike Men's CJ Elite 2 TD Football Cleat"}), support=0.010098176718092567, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bag Boy Beverage Holder'}), items_add=frozenset({"Nike Men's CJ Elite 2 TD Football Cleat"}), confidence=0.8059701492537313, lift=1.710627256651847)]),
 RelationRecord(items=frozenset({"Nike Men's Dri-FIT Victory Golf Polo", 'Bag Boy Beverage Holder'}), support=0.010472183263207107, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bag Boy Beverage Holder'}), items_add=frozenset({"Nike Men's Dri-FIT Victory Golf Polo"}), confidence=0.8358208955223881, lift=1.8150465944389729)]),
 RelationRecord(items=frozenset({'Bag Boy Beverage Holder', "O'Brien Men's Neoprene Life Vest"}), support=0.010004675081813932, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bag Boy Beverage Holder'}), items_add=frozenset({"O'Brien Men's Neoprene Life Vest"}), confidence=0.7985074626865671, lift=

### Corporate

In [70]:
g_df = df_dataco[df_dataco['Customer Segment'] == 'Corporate']
g_df

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Id,Customer Segment,Department Name,...,Order Item Quantity,Sales,Order Item Total,Order Region,Order State,Order Status,Product Name,Product Price,shipping date (DateOrders),Shipping Mode
4,2,4,134.210007,298.250000,Advance shipping,0,Sporting Goods,19489,Corporate,Fitness,...,1,327.750000,298.250000,Oceania,Queensland,PENDING_PAYMENT,Smart watch,327.750000,1/15/2018 11:24,Standard Class
7,2,1,68.430000,285.140015,Late delivery,1,Sporting Goods,19486,Corporate,Fitness,...,1,327.750000,285.140015,Eastern Asia,Guangdong,PROCESSING,Smart watch,327.750000,1/15/2018 10:21,First Class
8,3,2,133.720001,278.589996,Late delivery,1,Sporting Goods,19485,Corporate,Fitness,...,1,327.750000,278.589996,Eastern Asia,Guangdong,CLOSED,Smart watch,327.750000,1/16/2018 10:00,Second Class
9,2,1,132.149994,275.309998,Late delivery,1,Sporting Goods,19484,Corporate,Fitness,...,1,327.750000,275.309998,Eastern Asia,Guangdong,CLOSED,Smart watch,327.750000,1/15/2018 9:39,First Class
10,6,2,130.580002,272.029999,Shipping canceled,0,Sporting Goods,19483,Corporate,Fitness,...,1,327.750000,272.029999,Eastern Asia,Tokio,SUSPECTED_FRAUD,Smart watch,327.750000,1/19/2018 9:18,Second Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180508,4,2,85.000000,339.980011,Late delivery,1,Fishing,8387,Corporate,Fan Shop,...,1,399.980011,339.980011,Eastern Asia,Hubei,COMPLETE,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/20/2016 7:10,Second Class
180509,3,4,0.000000,335.980011,Advance shipping,0,Fishing,7,Corporate,Fan Shop,...,1,399.980011,335.980011,Eastern Asia,Hubei,PENDING_PAYMENT,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/19/2016 6:49,Standard Class
180510,3,4,165.990005,331.980011,Advance shipping,0,Fishing,7,Corporate,Fan Shop,...,1,399.980011,331.980011,Eastern Asia,Hubei,PENDING_PAYMENT,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/19/2016 6:49,Standard Class
180515,3,2,-613.770019,395.980011,Late delivery,1,Fishing,9141,Corporate,Fan Shop,...,1,399.980011,395.980011,Eastern Asia,Osaka,COMPLETE,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/19/2016 1:34,Second Class


In [71]:
cust_level_g = g_df[["Customer Id", "Product Name"]].sort_values(by = "Customer Id", ascending = False) ## Selecting only required variables for modelling
cust_level_g['Product Name'] = cust_level_g['Product Name'].str.strip() # Removing white spaces if any
cust_level_g

,Customer Id,Product Name
17863,20757,Fighting video games
52147,20756,Toys
86254,20746,Summer dresses
86606,20737,Summer dresses
41505,20736,Summer dresses
...,...,...
68442,7,Nike Men's Free 5.0+ Running Shoe
179873,7,Field & Stream Sportsman 16 Gun Fire Safe
40583,7,Nike Men's CJ Elite 2 TD Football Cleat
66987,7,Nike Men's Dri-FIT Victory Golf Polo


In [72]:
transactions_g = [a[1]['Product Name'].tolist() for a in list(cust_level_g.groupby(['Customer Id']))] ## Combing all the items in list format for each cutomer

In [73]:
transactions_g

[['Field & Stream Sportsman 16 Gun Fire Safe',
  "Diamondback Women's Serene Classic Comfort Bi",
  'Field & Stream Sportsman 16 Gun Fire Safe',
  'Field & Stream Sportsman 16 Gun Fire Safe',
  "Nike Men's Dri-FIT Victory Golf Polo",
  "Nike Men's Free 5.0+ Running Shoe",
  "Diamondback Women's Serene Classic Comfort Bi",
  "Nike Men's Free 5.0+ Running Shoe",
  'Team Golf New England Patriots Putter Grip',
  "Nike Men's Dri-FIT Victory Golf Polo",
  "Nike Men's Dri-FIT Victory Golf Polo",
  "Nike Men's CJ Elite 2 TD Football Cleat",
  "Nike Men's CJ Elite 2 TD Football Cleat",
  "O'Brien Men's Neoprene Life Vest",
  "O'Brien Men's Neoprene Life Vest",
  "Nike Men's CJ Elite 2 TD Football Cleat",
  'Field & Stream Sportsman 16 Gun Fire Safe',
  "Nike Men's Free 5.0+ Running Shoe",
  'Field & Stream Sportsman 16 Gun Fire Safe',
  "Nike Men's CJ Elite 2 TD Football Cleat",
  "Nike Men's Dri-FIT Victory Golf Polo",
  "Nike Men's Free 5.0+ Running Shoe"],
 ["Nike Men's Dri-FIT Victory Golf

In [74]:
#Importing and Setting the apriori options
#1. Minimum Support
#2. Minimum Confidence
#3. Minimum Lift
#4. Lenght = the number of items

In [75]:
from apyori import apriori ## Importing apriori package
rules = apriori(transactions = transactions_g, min_support = 0.01, min_confidence = 0.3, min_lift = 1.12, min_length = 2, max_length = 2) ## Model Creation

In [76]:
results = list(rules)

In [77]:
## Creating user-defined function for arranging the results obtained from model into readable format

def inspect(results):
    x         = [tuple(result[2][0][0])[0] for result in results]
    y         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(x, y, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['X', 'Y', 'Support', 'Confidence', 'Lift'])

In [78]:
resultsinDataFrame.nlargest(n=40, columns="Support") ## Showing best possible scenarios

,X,Y,Support,Confidence,Lift
159,Nike Men's CJ Elite 2 TD Football Cleat,Perfect Fitness Perfect Rip Deck,0.397339,0.841766,1.735551
198,Nike Men's Dri-FIT Victory Golf Polo,Perfect Fitness Perfect Rip Deck,0.390607,0.849129,1.750732
153,Nike Men's CJ Elite 2 TD Football Cleat,Nike Men's Dri-FIT Victory Golf Polo,0.377144,0.798981,1.736880
234,O'Brien Men's Neoprene Life Vest,Perfect Fitness Perfect Rip Deck,0.376823,0.847818,1.748030
157,Nike Men's CJ Elite 2 TD Football Cleat,O'Brien Men's Neoprene Life Vest,0.365443,0.774194,1.741866
81,Field & Stream Sportsman 16 Gun Fire Safe,Perfect Fitness Perfect Rip Deck,0.356628,0.847296,1.746954
196,Nike Men's Dri-FIT Victory Golf Polo,O'Brien Men's Neoprene Life Vest,0.353903,0.769338,1.730941
72,Field & Stream Sportsman 16 Gun Fire Safe,Nike Men's CJ Elite 2 TD Football Cleat,0.352300,0.837014,1.773220
257,Pelican Sunstream 100 Kayak,Perfect Fitness Perfect Rip Deck,0.339477,0.857837,1.768687
75,Field & Stream Sportsman 16 Gun Fire Safe,Nike Men's Dri-FIT Victory Golf Polo,0.338356,0.803884,1.747538


In [41]:
#result2 = resultsinDataFrame[resultsinDataFrame['X'] == "Nike Men's CJ Elite 2 TD Football Cleat"]
#result2.nlargest(n=5, columns='Support')

In [42]:
results

[RelationRecord(items=frozenset({'Bag Boy Beverage Holder', "Diamondback Women's Serene Classic Comfort Bi"}), support=0.01154031094726719, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bag Boy Beverage Holder'}), items_add=frozenset({"Diamondback Women's Serene Classic Comfort Bi"}), confidence=0.7422680412371134, lift=1.974844481568593)]),
 RelationRecord(items=frozenset({'Bag Boy Beverage Holder', 'Field & Stream Sportsman 16 Gun Fire Safe'}), support=0.0123417214297163, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bag Boy Beverage Holder'}), items_add=frozenset({'Field & Stream Sportsman 16 Gun Fire Safe'}), confidence=0.7938144329896907, lift=1.8859894316156436)]),
 RelationRecord(items=frozenset({'Bag Boy Beverage Holder', "Nike Men's CJ Elite 2 TD Football Cleat"}), support=0.012822567719185766, ordered_statistics=[OrderedStatistic(items_base=frozenset({'Bag Boy Beverage Holder'}), items_add=frozenset({"Nike Men's CJ Elite 2 TD Football Cleat"}), con

### Home Office

In [43]:
s_df = df_dataco[df_dataco['Customer Segment'] == 'Home Office']
s_df

,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Name,Customer Id,Customer Segment,Department Name,...,Order Item Quantity,Sales,Order Item Total,Order Region,Order State,Order Status,Product Name,Product Price,shipping date (DateOrders),Shipping Mode
3,3,4,22.860001,304.809998,Advance shipping,0,Sporting Goods,19490,Home Office,Fitness,...,1,327.750000,304.809998,Oceania,Queensland,COMPLETE,Smart watch,327.750000,1/16/2018 11:45,Standard Class
6,2,1,95.180000,288.420013,Late delivery,1,Sporting Goods,19487,Home Office,Fitness,...,1,327.750000,288.420013,Eastern Asia,Guangdong,COMPLETE,Smart watch,327.750000,1/15/2018 10:42,First Class
41,1,0,135.690002,294.980011,Late delivery,1,Sporting Goods,19452,Home Office,Fitness,...,1,327.750000,294.980011,Southeast Asia,Singapur,COMPLETE,Smart watch,327.750000,1/13/2018 10:27,Same Day
42,4,4,132.669998,288.420013,Shipping on time,0,Sporting Goods,19451,Home Office,Fitness,...,1,327.750000,288.420013,Southeast Asia,Sumatra Septentrional,CLOSED,Smart watch,327.750000,1/16/2018 22:05,Standard Class
43,3,4,119.760002,285.140015,Advance shipping,0,Sporting Goods,19450,Home Office,Fitness,...,1,327.750000,285.140015,Southeast Asia,Sumatra Septentrional,PENDING_PAYMENT,Smart watch,327.750000,1/15/2018 21:44,Standard Class
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180502,2,4,23.809999,377.980011,Advance shipping,0,Fishing,5424,Home Office,Fan Shop,...,1,399.980011,377.980011,Oceania,Queensland,PENDING_PAYMENT,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/18/2016 15:14,Standard Class
180503,2,4,125.730003,371.980011,Advance shipping,0,Fishing,5424,Home Office,Fan Shop,...,1,399.980011,371.980011,Oceania,Queensland,PENDING_PAYMENT,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/18/2016 15:14,Standard Class
180512,6,4,86.400002,319.980011,Late delivery,1,Fishing,7396,Home Office,Fan Shop,...,1,399.980011,319.980011,Eastern Asia,Sichuan,COMPLETE,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/22/2016 6:07,Standard Class
180513,3,4,119.989998,299.989990,Advance shipping,0,Fishing,3080,Home Office,Fan Shop,...,1,399.980011,299.989990,Eastern Asia,Shanghái,PENDING_PAYMENT,Field & Stream Sportsman 16 Gun Fire Safe,399.980011,1/19/2016 5:04,Standard Class


In [44]:
cust_level_s = s_df[["Order Id", "Product Name"]].sort_values(by = "Order Id", ascending = False) ## Selecting only required variables for modelling
cust_level_s['Product Name'] = cust_level_s['Product Name'].str.strip() # Removing white spaces if any
cust_level_s

,Order Id,Product Name
62641,77198,Summer dresses
142093,77197,Summer dresses
97707,77196,Summer dresses
168609,77195,Summer dresses
142110,77189,Summer dresses
...,...,...
141056,12,Nike Men's Free 5.0+ Running Shoe
63764,4,Team Golf New England Patriots Putter Grip
95938,4,Nike Men's Dri-FIT Victory Golf Polo
114915,4,O'Brien Men's Neoprene Life Vest


In [45]:
transactions_s = [a[1]['Product Name'].tolist() for a in list(cust_level_s.groupby(['Order Id']))] ## Combing all the items in list format for each cutomer

In [46]:
transactions_s

[['Team Golf New England Patriots Putter Grip',
  "Nike Men's Dri-FIT Victory Golf Polo",
  "O'Brien Men's Neoprene Life Vest",
  'Perfect Fitness Perfect Rip Deck'],
 ["Nike Men's Dri-FIT Victory Golf Polo",
  "Nike Women's Legend V-Neck T-Shirt",
  "Diamondback Women's Serene Classic Comfort Bi",
  "O'Brien Men's Neoprene Life Vest",
  "Nike Men's Free 5.0+ Running Shoe"],
 ["Diamondback Women's Serene Classic Comfort Bi"],
 ['Field & Stream Sportsman 16 Gun Fire Safe'],
 ['Field & Stream Sportsman 16 Gun Fire Safe',
  'Pelican Sunstream 100 Kayak',
  "O'Brien Men's Neoprene Life Vest",
  "Nike Men's CJ Elite 2 TD Football Cleat",
  "Nike Men's CJ Elite 2 TD Football Cleat"],
 ["O'Brien Men's Neoprene Life Vest",
  'Bridgestone e6 Straight Distance NFL Tennesse'],
 ['Pelican Sunstream 100 Kayak',
  "Nike Men's CJ Elite 2 TD Football Cleat",
  'Clicgear 8.0 Shoe Brush',
  'Field & Stream Sportsman 16 Gun Fire Safe',
  "Nike Men's Dri-FIT Victory Golf Polo"],
 ['Perfect Fitness Perfect

In [ ]:
#Importing and Setting the apriori options
#1. Minimum Support
#2. Minimum Confidence
#3. Minimum Lift
#4. Lenght = the number of items

In [47]:
from apyori import apriori ## Importing apriori package
rules = apriori(transactions = transactions_s, min_support = 0.01, min_confidence = 0.1, min_lift = 1.12, min_length = 2, max_length = 2) ## Model Creation

In [48]:
results = list(rules)

In [49]:
## Creating user-defined function for arranging the results obtained from model into readable format

def inspect(results):
    x         = [tuple(result[2][0][0])[0] for result in results]
    y         = [tuple(result[2][0][1])[0] for result in results]
    supports    = [result[1] for result in results]
    confidences = [result[2][0][2] for result in results]
    lifts       = [result[2][0][3] for result in results]
    return list(zip(x, y, supports, confidences, lifts))
resultsinDataFrame = pd.DataFrame(inspect(results), columns = ['X', 'Y', 'Support', 'Confidence', 'Lift'])

In [50]:
resultsinDataFrame.nlargest(n=30, columns="Support") ## Showing best possible scenarios

,X,Y,Support,Confidence,Lift
1,Nike Men's CJ Elite 2 TD Football Cleat,Nike Men's Free 5.0+ Running Shoe,0.052985,0.186826,1.133567
0,Diamondback Women's Serene Classic Comfort Bi,Under Armour Girls' Toddler Spine Surge Runni,0.031247,0.166818,1.127792
